In [1]:
from biopandas.pdb import PandasPdb
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
import plotly.express as px
import plotly.graph_objects as go
from unsync import unsync

## Install `pdb_profiling`

```bash
pip install pdb-profiling
```

In [2]:
from pdb_profiling.fetcher.webfetch import UnsyncFetch
from pdb_profiling.utils import DisplayPDB, iter_first
from pdb_profiling.processers.pdbe.api import *
from pdb_profiling.processers.pdbe.record import *

ProcessPDBe.use_existing = True  # Use Existing Handled PDBe API Results (e.g. tsv format results)
ProcessPDBe.init_logger()  # Init PDBe API Logger
UnsyncFetch.use_existing = True  # Use Existing API Results (e.g. json format results downloaded from web)
UnsyncFetch.init_setting(ProcessPDBe.logger)  # Init WebFetcher's Logger (pass it with PDBe API Logger)
PDB.set_web_semaphore(30)  # Set WebFetcher's Semaphore
PDB.set_folder('./')  # Set Folder that store downloaded and handled files

## Prepare Functions

In [3]:
def getModelIndex(ppdb: PandasPdb):
    '''
    Attempt to get seperate dataframe of different model 
    by getting corresponding line_idx from 'OHTERS' dataframe
    '''
    for start_r, end_r in zip(
        ppdb.df['OTHERS'].query('record_name == "MODEL"')[['entry', 'line_idx']].to_dict('record'),
        ppdb.df['OTHERS'].query('record_name == "ENDMDL"')[['line_idx']].to_dict('record')):
        yield dict(
            MODEL=start_r['entry'].strip(), 
            start_line_idx=start_r['line_idx'],
            end_line_idx=end_r['line_idx'])

def get_rmsd(mol1, mol2):
    '''
    Calculate the Root Mean Squar Deviation/Error
    '''
    # return np.sqrt(((mol1 - mol2) ** 2).mean())
    return np.sqrt(np.mean(np.square(mol1 - mol2)))

@unsync
async def pipeline(pdb_id:str, chain_id:str, imposed_file_path:str, query_sites: Iterable[str]=[]):
    pdb_demo = PDB(pdb_id)
    
    mol_info_df = await pdb_demo.get_res2eec_df()
    info_tp = iter_first(mol_info_df, lambda x: x.chain_id == chain_id and x.molecule_type == 'polypeptide(L)')
    res_df = await pdb_demo.fetch_from_web_api('api/pdb/entry/residue_listing/', PDB.to_dataframe)
    reIndex_df = res_df[(res_df.entity_id == info_tp.entity_id) & (res_df.chain_id == info_tp.chain_id)][
        ['chain_id', 'residue_number', 'author_residue_number', 'author_insertion_code']]
    
    ppdb = PandasPdb().read_pdb(imposed_file_path)
    ppdb.df['ATOM']['residue_site'] = ppdb.df['ATOM'].residue_number.astype(str)+ppdb.df['ATOM'].insertion
    ppdb.df['ATOM'].rename(columns={'residue_number': 'author_residue_number', 'insertion': 'author_insertion_code'}, inplace=True)
    df_model = {record["MODEL"]: ppdb.df['ATOM'].query(
        f'line_idx >= {record["start_line_idx"]} & line_idx <= {record["end_line_idx"]}') for record in getModelIndex(ppdb)}
    
    df_model['1'] = df_model['1'].merge(reIndex_df, how='left')
    assert df_model['1'].residue_number.isnull().sum() == 0, f"Unexpected Cases: {df_model['1'][df_model['1'].residue_number.isnull()]}"
    df_model['2']['residue_number'] = df_model['2'].author_residue_number
    residue_number_intersection = sorted(set(df_model['1'].residue_number) & set(df_model['2'].residue_number))

    query_unit_df = pd.DataFrame({'residue_number': residue_number_intersection, 'atom_name': 'CA'})
    df_model_1_df = df_model['1'].merge(query_unit_df, how='inner')
    df_model_2_df = df_model['2'].merge(query_unit_df, how='inner')

    mol1 = df_model_1_df[['x_coord','y_coord','z_coord']].to_numpy()
    mol2 = df_model_2_df[['x_coord','y_coord','z_coord']].to_numpy()
    # ---subset---
    tree1 = KDTree(mol1)
    tree2 = KDTree(mol2)
    if len(query_sites) == 0:
        return (pdb_id, chain_id, imposed_file_path), dict(overall_rmsd=get_rmsd(mol1, mol2), sites_rmsd=None)
    
    df_model_1_sites_df = df_model_1_df.query(f'residue_site in {query_sites}')
    df_model_2_sites_df = df_model_2_df.query(f'residue_site in {query_sites}')


    res_index_1 = tree1.query_radius(
        df_model_1_sites_df[['x_coord','y_coord','z_coord']].to_numpy(), r=10)
    res_index_2 = tree2.query_radius(
        df_model_2_sites_df[['x_coord','y_coord','z_coord']].to_numpy(), r=10)
    res_indexes = [np.intersect1d(*i) for i in zip(res_index_1,res_index_2)]

    return (pdb_id, chain_id, imposed_file_path), dict(
        overall_rmsd=get_rmsd(mol1, mol2), 
        sites_rmsd=zip(df_model_1_sites_df.residue_site, (get_rmsd(mol1[res_index], mol2[res_index]) for res_index in res_indexes)))
            

In [4]:
%time pipeline('4v2f', 'A', '4V2FA_2XPWA_FATCAT.pdb').result()

Wall time: 410 ms


(('4v2f', 'A', '4V2FA_2XPWA_FATCAT.pdb'),
 {'overall_rmsd': 0.7241765068863248, 'sites_rmsd': None})

In [5]:
%time input_info, output_data = pipeline('4v2f', 'A', '4V2FA_2XPWA_FATCAT.pdb', ("141", "142")).result()
print(input_info, output_data)
print(tuple(output_data['sites_rmsd']))

Wall time: 465 ms
('4v2f', 'A', '4V2FA_2XPWA_FATCAT.pdb') {'overall_rmsd': 0.7241765068863248, 'sites_rmsd': <zip object at 0x00000205C0110F88>}
(('141', 0.37412670536419396), ('142', 0.3888843015602454))


In [6]:
# MultiPle Tasks
sites1, sites2, sites3 = ("33", "21"), ("141", "140", "142"), ("101", "102", "103")

tasks = (
    pipeline('1A5E', 'A', '1A5EA_1BD8A_FATCAT.pdb', sites1),
    pipeline('1A5E', 'A', '1A5EA_1MX4B_FATCAT.pdb', sites1),
    pipeline('1A5E', 'A', '1A5EA_1N11A_FATCAT.pdb', sites1),
    pipeline('4V2F', 'A', '4V2FA_2XPWA_FATCAT.pdb', sites2),
    pipeline('1RN1', 'C', '1RN1C_1FUSA_FATCAT.pdb', sites3)
)

res = UnsyncFetch.unsync_tasks(tasks).result()
dict(res)

100%|██████████| 5/5 [00:02<00:00,  2.40it/s]


{('1A5E', 'A', '1A5EA_1BD8A_FATCAT.pdb'): {'overall_rmsd': 4.256621864661923,
  'sites_rmsd': <zip at 0x205c00ade08>},
 ('4V2F', 'A', '4V2FA_2XPWA_FATCAT.pdb'): {'overall_rmsd': 0.7241765068863248,
  'sites_rmsd': <zip at 0x205c0101c48>},
 ('1A5E', 'A', '1A5EA_1MX4B_FATCAT.pdb'): {'overall_rmsd': 4.800521648782281,
  'sites_rmsd': <zip at 0x205c00ad548>},
 ('1RN1', 'C', '1RN1C_1FUSA_FATCAT.pdb'): {'overall_rmsd': 0.9899995904557556,
  'sites_rmsd': <zip at 0x205c03c9d48>},
 ('1A5E', 'A', '1A5EA_1N11A_FATCAT.pdb'): {'overall_rmsd': 5.697866133606135,
  'sites_rmsd': <zip at 0x205c049ef88>}}

In [7]:
for input_info, output_data in res:
    print(input_info, output_data['overall_rmsd'])
    print(dict(output_data['sites_rmsd']))
    print()

('1A5E', 'A', '1A5EA_1BD8A_FATCAT.pdb') 4.256621864661923
{'21': 0.718660353507625, '33': 1.3363094181779573}

('4V2F', 'A', '4V2FA_2XPWA_FATCAT.pdb') 0.7241765068863248
{'140': 0.3511797082691419, '141': 0.37412670536419396, '142': 0.3888843015602454}

('1A5E', 'A', '1A5EA_1MX4B_FATCAT.pdb') 4.800521648782281
{'21': 0.9438510750387753, '33': 1.2094294660431144}

('1RN1', 'C', '1RN1C_1FUSA_FATCAT.pdb') 0.9899995904557556
{'101': 0.44910441094452086, '102': 0.32285424988168665, '103': 0.3168644996068684}

('1A5E', 'A', '1A5EA_1N11A_FATCAT.pdb') 5.697866133606135
{'21': 1.498311224784687, '33': 0.7462497753920779}



In [8]:
# TODO: CHECK
input_info, output_data = pipeline('1A5E', 'A', '1A5EA_1BD8A_FATCAT.pdb', ('21',)).result()
print(input_info, output_data['overall_rmsd'])
print(dict(output_data['sites_rmsd']))
print()

input_info, output_data = pipeline('4V2F', 'A', '4V2FA_2XPWA_FATCAT.pdb', ('142',)).result()
print(input_info, output_data['overall_rmsd'])
print(dict(output_data['sites_rmsd']))
print()

input_info, output_data = pipeline('4V2F', 'A', '4V2FA_2XPWA_FATCAT.pdb', ('140',)).result()
print(input_info, output_data['overall_rmsd'])
print(dict(output_data['sites_rmsd']))
print()

('1A5E', 'A', '1A5EA_1BD8A_FATCAT.pdb') 4.256621864661923
{'21': 0.718660353507625}

('4V2F', 'A', '4V2FA_2XPWA_FATCAT.pdb') 0.7241765068863248
{'142': 0.3888843015602454}

('4V2F', 'A', '4V2FA_2XPWA_FATCAT.pdb') 0.7241765068863248
{'140': 0.3511797082691419}



## Explain `pipeline()` Step by Step

### 1. Get Residue-Listing Data Via `pdb_profiling` Package

In [9]:
pdb_demo = PDB('4v2f')
'''
TODO: Check PDB Entry Status

    * status_code: REL -> relase
    * status_code: OBS -> obsolete
'''
assert pdb_demo.status['status_code'] == 'REL', pdb_demo.status

#### NOTE $\downarrow$

1. `info_tp` contains corresponding entity_id that can be used to retrieve corresponding residue-listing data in the next step
2. Sometimes there may be multiple instances with the same chain_id (i.e. 5pti), so we need to get its entity_id to identify the protein chain with a particular chain_id. Otherwise one should use struct_asym_id.

In [10]:
# TODO: Get Polypeptide Instance with Specified `chain_id`
mol_info_df = pdb_demo.get_res2eec_df().result()
info_tp = iter_first(mol_info_df, lambda x: x.chain_id == 'A' and x.molecule_type == 'polypeptide(L)')
mol_info_df.T


,0
pdb_id,4v2f
entity_id,1
chain_id,A
struct_asym_id,A
struct_asym_id_in_assembly,A
ca_p_only,False
gene_name,"[""tetR""]"
in_chains,"[""A""]"
in_struct_asyms,"[""A""]"
length,207


In [11]:
# TODO: Get Polypeptide Instance with Specified `chain_id`
res_df = pdb_demo.fetch_from_web_api('api/pdb/entry/residue_listing/', PDB.to_dataframe).result()
sele_res_df = res_df[(res_df.entity_id == info_tp.entity_id) & (res_df.chain_id == info_tp.chain_id)][['chain_id', 'residue_number', 'author_residue_number', 'author_insertion_code']]
sele_res_df.head()

,chain_id,residue_number,author_residue_number,author_insertion_code
0,A,1,2,
1,A,2,3,
2,A,3,4,
3,A,4,5,
4,A,5,6,


### 2. Load Super-Imposed Structure via `PandasPdb`

<table>
<tr>
<td>
<img src="../docs/figs/4V2FA_2XPWA_FATCAT.PDB-4V2F.png" width="300em">
</td>
</tr>
<tr>
<td>
4V2FA_2XPWA_FATCAT.pdb
</td>
</tr>
</table>

In [12]:
ppdb = PandasPdb().read_pdb('4V2FA_2XPWA_FATCAT.pdb')
ppdb.df['ATOM']['residue_site'] = ppdb.df['ATOM'].residue_number.astype(str)+ppdb.df['ATOM'].insertion
ppdb.df['ATOM'].rename(columns={'residue_number': 'author_residue_number', 'insertion': 'author_insertion_code'}, inplace=True)
print(ppdb.df.keys())
ppdb.df['OTHERS']

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])


,record_name,entry,line_idx
0,TITLE,jFatCat_rigid 1.1 4V2FA.pdb vs. 4V2FA_2XPWA_1...,0
1,EXPDTA,"NMR, 2 STRUCTURES",1
2,MODEL,1,2
3,ENDMDL,,1544
4,MODEL,2,1545
5,ENDMDL,,3187


In [13]:
df_model = {record["MODEL"]: ppdb.df['ATOM'].query(
    f'line_idx >= {record["start_line_idx"]} & line_idx <= {record["end_line_idx"]}') for record in getModelIndex(ppdb)}

In [14]:
# NOTE: append `residue_number` columns to df_model['1']
df_model['1'] = df_model['1'].merge(sele_res_df, how='left')
# NOTE: make sure that there is no unmap residue
assert df_model['1'].residue_number.isnull().sum() == 0, "Unexpected Cases"
# NOTE: for modelled structure, just copy the original column
df_model['2']['residue_number'] = df_model['2'].author_residue_number
# NOTE: get intersection of residue numbers
residue_number_intersection = sorted(set(df_model['1'].residue_number) & set(df_model['2'].residue_number))

### 3. Calculate RMSD of Full SuperImposed Structure (CA ATOM only)

In [15]:
query_unit_df = pd.DataFrame({'residue_number': residue_number_intersection, 'atom_name': 'CA'})
df_model_1_df = df_model['1'].merge(query_unit_df, how='inner')
df_model_2_df = df_model['2'].merge(query_unit_df, how='inner')

mol1 = df_model_1_df[['x_coord','y_coord','z_coord']].to_numpy()
mol2 = df_model_2_df[['x_coord','y_coord','z_coord']].to_numpy()
# NOTE: calculate RMSD
%time print(get_rmsd(mol1, mol2))

0.7241765068863248
Wall time: 997 µs


### 4. Visualize Full SuperImposed Structure (CA ATOM only)

In [16]:
df_to_plot_1 = df_model['1'].query('atom_name == "CA"').reset_index(drop=True); df_to_plot_1['Source'] = 'MODEL 1'

df_to_plot_2 = df_model['2'].query('atom_name == "CA"').reset_index(drop=True); df_to_plot_2['Source'] = 'MODEL 2'

df_to_plot_1['size'] = 1
df_to_plot_2['size'] = 1

df_to_plot = pd.concat([df_to_plot_1,df_to_plot_2],ignore_index=True, sort=False)

# TODO: plot it with `plotly`
fig = px.scatter_3d(df_to_plot, x='x_coord', y='y_coord', z='z_coord',
              color='Source',size='size',size_max=10, template='plotly_dark')
fig.update_traces(mode='markers+lines')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

### 5. Get all residues within the given radius from the source residue

* by getting residue's index

#### K-D Tree

> In computer science, a k-d tree (short for k-dimensional tree) is a space-partitioning data structure for organizing points in a k-dimensional space. k-d trees are a useful data structure for several applications, such as searches involving a multidimensional search key (e.g. range searches and nearest neighbor searches). k-d trees are a special case of binary space partitioning trees.

> Wikipedia contributors. (2020, September 9). K-d tree. In Wikipedia, The Free Encyclopedia. Retrieved 13:47, September 18, 2020, from https://en.wikipedia.org/w/index.php?title=K-d_tree&oldid=977572387

In [17]:
tree1 = KDTree(mol1)
tree2 = KDTree(mol2)
# Once built, multiple uses
query_sites = ["141","142"]

df_model_1_sites_df = df_model_1_df.query(f'residue_site in {query_sites}')
df_model_2_sites_df = df_model_2_df.query(f'residue_site in {query_sites}')


res_index_1 = tree1.query_radius(
    df_model_1_sites_df[['x_coord','y_coord','z_coord']].to_numpy(), r=10)

res_index_2 = tree2.query_radius(
    df_model_2_sites_df[['x_coord','y_coord','z_coord']].to_numpy(), r=10)

res_indexes = [np.intersect1d(*i) for i in zip(res_index_1,res_index_2)]

for query_site,i,j,k in zip(query_sites,res_index_1, res_index_2, res_indexes):
    print('query_site:', query_site)
    print('from MODEL 1:', sorted(i))
    print('from MODEL 2:', sorted(j))
    print('from interse:', sorted(k))
    print()

query_site: 141
from MODEL 1: [81, 82, 84, 85, 88, 95, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 170, 171, 173, 174]
from MODEL 2: [81, 84, 85, 88, 89, 94, 95, 96, 98, 99, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146]
from interse: [81, 84, 85, 88, 95, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144]

query_site: 142
from MODEL 1: [81, 84, 85, 88, 95, 96, 98, 99, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146]
from MODEL 2: [95, 96, 99, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 170]
from interse: [95, 96, 99, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146]



### 6. Calculate RMSD of Particular Subset of SuperImposed Structure (CA ATOM only)

In [19]:
for res_index in res_indexes:
    print(get_rmsd(mol1[res_index], mol2[res_index]))

0.37412670536419396
0.3888843015602454


In [20]:
dict(zip(df_model_1_sites_df.residue_site, (get_rmsd(mol1[res_index], mol2[res_index]) for res_index in res_indexes)))

{'141': 0.37412670536419396, '142': 0.3888843015602454}

### 7. Visualize Particular Subset of SuperImposed Structure (CA ATOM only)

* get subset dataframe: e.g. `df_to_plot_1.loc[res_index]`

In [21]:
res_index = res_indexes[1]

df_to_plot_1.size = 1;df_to_plot_1.loc[res_index, 'size'] = 5

df_to_plot_2.size = 1;df_to_plot_2.loc[res_index, 'size'] = 5

df_to_plot = pd.concat([df_to_plot_1,df_to_plot_2],ignore_index=True, sort=False)

fig = px.scatter_3d(df_to_plot, x='x_coord', y='y_coord', z='z_coord',
              color='Source',size='size',template='plotly_dark')

fig.update_traces(mode='markers+lines')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()